# **Regression Using Decision Trees**

Decision Trees are a versatile algorithm that can be used for both classification and regression tasks. Although they are popularly used for classification, they are also applicable for regression analysis.

In both classification and regression trees, the dataset is split into subsets, and then the impurity measure is calculated to find the best possible split. In the case of classification trees, we calculate the entropy or gini index for each subset, and then calculate the information gain.

In regression trees, as we deal with continuous values, we use a different metric to calculate the impurity measure. We use a technique called variance reduction, where we calculate the variance of all possible splits of each subset and then calculate the Mean Squared Error (MSE) for each split. We select the split that has a lower MSE, and this process continues until we get satisfactory results. Finally, we average the values of the leaf node to make the prediction.

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing

data = fetch_california_housing()

In [2]:
X = data['data']
y = data['target']

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [14]:
def best_split(feature_values, target_values):
    unique_values = np.unique(feature_values)
    best_split_value = None
    best_mse = float('inf')

    sorted_indices = np.argsort(feature_values)
    sorted_feature_values = feature_values[sorted_indices]
    sorted_target_values = target_values[sorted_indices]

    for i in range(len(unique_values) - 1):
        split_value = (unique_values[i] + unique_values[i+1]) / 2
        left_indices = np.where(sorted_feature_values <= split_value)[0]
        right_indices = np.where(sorted_feature_values > split_value)[0]

        left_mean = np.mean(sorted_target_values[left_indices])
        right_mean = np.mean(sorted_target_values[right_indices])

        mse = np.mean((sorted_target_values[left_indices] - left_mean) ** 2) + \
        np.mean((sorted_target_values[right_indices] - right_mean) ** 2)

        if mse < best_mse:
            best_mse = mse
            best_split_value = split_value

    return best_split_value, best_mse

In [5]:

# The Decision Tree Node
class Node:

    def __init__(self, feature_index = None, threshold = None, left = None, right = None, value = None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value


class DecisionTreeRegressor:

    def __init__(self, max_depth = None):
        self.max_depth = max_depth
        self.tree = None

    # Calculate Mean Squared Error
    def _calculate_mse(self, y_left, y_right):
        mse_left = np.mean((y_left - np.mean(y_left)) ** 2) # calculating the MSE of left subset
        mse_right = np.mean((y_right - np.mean(y_right)) ** 2) # calculating the MSE of right subset
        return mse_left + mse_right # Total MSE

    def _find_best_split(self, X, y):
        best_mse = float('inf')
        best_feature_index = None
        best_threshold = None

        n_samples, n_features = X.shape

        # Finding best splits by checking each features
        for feature_index in range(n_features):

            sorted_indices = np.argsort(X[:, feature_index]) # Finding the index that sorts the features
            sorted_values = X[sorted_indices, feature_index] # Taking sorted values using sorting indices
            sorted_labels = y[sorted_indices] # Sorted target values

            unique_values = np.unique(sorted_values)
            average_thresholds = (unique_values[:-1] + unique_values[1:]) / 2

            for threshold in average_thresholds:
                # Values that are smaller than threshold goes to the left of the tree
                left_indices = np.where(X[:, feature_index] <= threshold)[0]

                # Values that are larger than threshold goes to the right of the tree
                right_indices = np.where(X[:, feature_index] > threshold)[0]

                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue

                y_left = y[left_indices]
                y_right = y[right_indices]

                # Calculating MSE for each splits on every features
                mse = self._calculate_mse(y_left, y_right)

                # Updating MSE
                if mse < best_mse:
                    best_mse = mse
                    best_feature_index = feature_index
                    best_threshold = threshold

        return best_feature_index, best_threshold


    def _build_tree(self, X, y, depth):
        n_samples, n_features = X.shape

        # Recursion base case, if the max depth is reached or samples in the leaf node is one,
        if depth == self.max_depth or n_samples == 1 or np.all(y == y[0]):
            return Node(value = np.mean(y))

        # Finding the best split
        best_feature_index, best_threshold = self._find_best_split(X, y)
        left_indices = np.where(X[:, best_feature_index] <= best_threshold)[0]
        right_indices = np.where(X[:, best_feature_index] > best_threshold)[0]

        # Building the tree
        left_subtree = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        right_subtree = self._build_tree(X[right_indices], y[right_indices], depth + 1)

        return Node(feature_index = best_feature_index, threshold = best_threshold, left = left_subtree, right = right_subtree)

    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth = 0)

    def _predict_sample(self, node, sample):
        if node.value is not None:
            return node.value

        if sample[node.feature_index] <= node.threshold:
            return self._predict_sample(node.left, sample)
        else:
            return self._predict_sample(node.right, sample)

    def predict(self, X):
        predictions = []
        for sample in X:
            predictions.append(self._predict_sample(self.tree, sample))

        return np.array(predictions)

In [15]:
dtr = DecisionTreeRegressor()
dtr.fit(X_train[:200], y_train[:200])

In [16]:
prediction = dtr.predict(X_test[:100])

In [17]:
from sklearn.metrics import mean_squared_error

mean_squared_error(prediction, y_test[:100])

1.249411709207